In [1]:
!wget https://downloads.rclone.org/v1.50.1/rclone-v1.50.1-linux-amd64.deb
!apt install ./rclone-v1.50.1-linux-amd64.deb

--2024-12-06 08:26:02--  https://downloads.rclone.org/v1.50.1/rclone-v1.50.1-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11917114 (11M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.50.1-linux-amd64.deb’

rclone-v1.50.1-linu 100%[===================>]  11.36M  5.39MB/s    in 2.1s    

2024-12-06 08:26:05 (5.39 MB/s) - ‘rclone-v1.50.1-linux-amd64.deb’ saved [11917114/11917114]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.50.1-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 0 B/11.9 MB of archives.
After this operation, 36.1 MB of additional disk space will be used.
Get

In [6]:
!rclone config

2024/12/06 08:26:52 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found - make a new one
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n
name> onedrive
Type of storage to configure.
Enter a string value. Press Enter for the default ("").
Choose a number from below, or type in your own value
 1 / 1Fichier
   \ "fichier"
 2 / Alias for an existing remote
   \ "alias"
 3 / Amazon Drive
   \ "amazon cloud drive"
 4 / Amazon S3 Compliant Storage Provider (AWS, Alibaba, Ceph, Digital Ocean, Dreamhost, IBM COS, Minio, etc)
   \ "s3"
 5 / Backblaze B2
   \ "b2"
 6 / Box
   \ "box"
 7 / Cache a remote
   \ "cache"
 8 / Citrix Sharefile
   \ "sharefile"
 9 / Dropbox
   \ "dropbox"
10 / Encrypt/Decrypt a remote
   \ "crypt"
11 / FTP Connection
   \ "ftp"
12 / Google Cloud Storage (this is not Google Drive)
   \ "google cloud storage"
13 / Google Drive
   \ "drive"
14 / Google Photos
   \ "google photos"
15 / Hubic
   \ "hubic"
16 

In [7]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


In [8]:
%cd /content/onedrive

/content/onedrive


### Connect to drive and import libraries





In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import re

from scipy.special import softmax
import numpy as np
os.chdir('/content/drive/MyDrive/Fairness_NLP/')

In [3]:
import re

def extract_text(input_string):
  # Check if the input is a string
    if not isinstance(input_string, str):
        print(input_string)
        raise ValueError("Expected a string input")
    # Use regex to find all text within parentheses and remove the text itself
    matches = re.findall(r'\((.*?)\)', input_string)

    if not matches:
        return input_string

    # Join the extracted texts with a space and return them
    return ' '.join(matches)

# Below helper function creates question-answer pairs (without filtering)
def create_question_answer_pairs(interview):
    question_answer_pairs = []
    current_question = []
    current_response = []

    for index, row in interview.iterrows():
        row['value'] = extract_text(str(row['value']))
        if row['speaker'] == "Ellie":
            # If there's an existing question and response, store the pair
            if current_question and current_response:

                question_answer_pairs.append({
                    'question': " ".join(current_question),
                    'answer': ". ".join(current_response)
                })
                current_response = []  # Reset responses for the next question
                current_question = []  # Reset question for the next batch

            # Add the new question or follow-up from Ellie to the current question
            current_question.append(str(row['value']))

        elif row['speaker'] == "Participant" and current_question:
            current_response.append(str(row['value']))

    # Add the last question-answer pair if it exists
    if current_question and current_response:

        question_answer_pairs.append({
            'question': " ".join(current_question),
            'answer': ". ".join(current_response)
        })

    return pd.DataFrame(question_answer_pairs, columns=['question', 'answer'])

In [4]:
# Function to create chunks of QA pairs with overlaps
def chunk_qa_pairs(df, max_tokens=80, max_overlap_tokens=40):
    chunks = []
    current_chunk = []
    current_chunk_word_count = 0

    # Combine questions and answers
    qa_pairs = [f"Interviewer: {row['question']} Interviewee: {row['answer']}" for _, row in df.iterrows()]

    for pair in qa_pairs:
        # Count words in the current pair
        pair_word_count = len(pair.split())

        # Check if adding this pair exceeds the max tokens
        if current_chunk_word_count + pair_word_count > max_tokens:
            # Save the current chunk
            chunks.append(" ".join(current_chunk))

            # Prepare for the next chunk
            # Determine overlap (complete QA pairs)
            overlap = []
            overlap_word_count = 0

            # Start from the last added complete pairs until it hits the token limit
            for qa in reversed(current_chunk):
                overlap_word_count += len(qa.split())
                if overlap_word_count >= max_overlap_tokens:
                    break
                overlap.append(qa)

            # Reverse to maintain original order
            overlap.reverse()

            # Start new chunk with overlap
            current_chunk = overlap
            current_chunk_word_count = sum(len(q.split()) for q in current_chunk)

        # Add the current pair to the chunk
        current_chunk.append(pair)
        current_chunk_word_count += pair_word_count

    # Add the last chunk if it has content
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [ ]:
!pip install openpyxl

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Create dictionary of {participant_id : PHQ_Binary}
id_depression_label_map = {}
all_ids = set()
sheet_name = 'Metadata_mapping'
file_path = 'DAIC demographc data.xlsx'
data_csv = pd.read_excel(file_path, sheet_name=sheet_name)
for i in range(len(data_csv['Participant_ID'])):
    id_depression_label_map[data_csv['Participant_ID'][i]] = data_csv['PHQ_Binary'][i]
    all_ids.add(data_csv['Participant_ID'][i])


In [6]:
import pickle
with open('/content/drive/MyDrive/Fairness_NLP/RULE_BASED/adjective_embeddings.pkl', 'rb') as file:
    true_dict = pickle.load(file)

In [8]:
!pip install spacy
import spacy

In [9]:
# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Function to find adjectives in a sentence using spaCy
def extract_adjectives_spacy(sentence):
    doc = nlp(sentence)
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    return adjectives

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

def get_rule_based_prediction(chunk, parameters, max_len):
  adj_count = 0
  adj_depressed = 0

  adjectives = extract_adjectives_spacy(chunk)
  if not adjectives:
      return 0

  encoded_text = tokenizer(chunk, return_tensors='pt', max_length=max_len, truncation=True, padding='max_length').to(device)

  # Get embeddings from the model
  with torch.no_grad():
      outputs = model(**encoded_text)

  for adj in adjectives:
      # Find the index of each adjective in the tokenized context
      adj_tokens = tokenizer.tokenize(adj)
      adj_token_id = tokenizer.convert_tokens_to_ids(adj_tokens[0])  # ID of the first token of the adjective
      adj_index = (encoded_text.input_ids[0] == adj_token_id).nonzero(as_tuple=True)[0]  # Find position of token in input

      if len(adj_index) > 0:  # Ensure adjective token was found
          adj_embedding = outputs.last_hidden_state[0, adj_index[0], :].cpu().numpy()

          if adj in true_dict:
              adj_count+=1
              true_embedding = true_dict[adj]
              similarity = cosine_similarity([adj_embedding], [true_embedding])[0][0]

              if similarity > parameters[0]:
                adj_depressed+=1

  try:
    if (adj_depressed/adj_count) > parameters[1]:
      return 1
    else:
      return 0
  except Exception as exp:
      return 0


In [27]:
import torch
from transformers import AutoTokenizer, AutoModel
access_token = "TOKEN"
model_name = "mental/mental-roberta-base"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModel.from_pretrained(model_name, token=access_token).to(device)


max_len = 510
parameters = [0.75, 0.50]

def collect_train_test_data(directory):
    X_train = []
    Y_train = []
    attention_masks = []
    rule_predictions = [] # collect rule based predictions

    for filename in os.listdir(directory):
      if filename.endswith(".csv"):
        interview_id = re.findall(r'\d+', filename)[0]
        print(interview_id)

        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, delimiter='\t')
        df.drop(columns=['start_time', 'stop_time'], axis=1, inplace=True)
        df.fillna('', inplace=True)

        # Step 1: Create QA Pair
        qa_df = create_question_answer_pairs(df)
        chunks = chunk_qa_pairs(qa_df, max_tokens=300, max_overlap_tokens=80)
        for idx, chunk in enumerate(chunks):
            encoded_text = tokenizer(chunk, add_special_tokens=True, max_length=max_len, truncation=True, padding="max_length")
            input_ids = encoded_text['input_ids']
            attention_mask = encoded_text['attention_mask']
            rule_prediction = get_rule_based_prediction(chunk, parameters, max_len)

            # Convert to torch tensors and add to the lists
            input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
            attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
            label_tensor = torch.tensor(id_depression_label_map[int(interview_id)], dtype=torch.long)
            rule_prediction_tensor = torch.tensor(rule_prediction, dtype=torch.long)

            X_train.append(input_ids_tensor)
            attention_masks.append(attention_mask_tensor)
            Y_train.append(label_tensor)
            rule_predictions.append(rule_prediction_tensor)

    X_train = torch.stack(X_train)
    Y_train = torch.stack(Y_train)
    attention_masks = torch.stack(attention_masks)
    rule_predictions = torch.stack(rule_predictions)
    return X_train, Y_train, attention_masks, rule_predictions





Using device: cuda


Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:

# Load the data
directory = '/content/drive/MyDrive/Fairness_NLP/Dataset'
X_train, Y_train, attention_masks, rule_predictions = collect_train_test_data(directory)


345
305
309
306
341
372
364
356
369
381
360
379
350
338
362
352
335
304
339
300
331
358
329
333
355
371
319
346
344
367
316
337
328
314
324
383
302
326
349
353
327
340
365
370
301
378
347
336
334
307
325
373
308
323
343
363
366
320
354
315
310
322
357
312
311
351
303
359
361
318
313
317
332
321
330
368
348
384
416
425
374
466
475
465
485
472
422
387
426
471
392
396
476
479
447
440
415
431
424
487
488
432
437
409
454
382
412
397
492
399
433
385
450
441
491
478
436
453
449
484
400
411
467
375
446
410
413
469
395
404
443
489
388
463
434
391
470
457
429
452
464
448
389
386
390
402
407
468
481
423
439
393
490
474
486
376
403
459
418
417
405
483
401
428
456
461
435
427
406
421
377
482
473
408
455
477
420
380
430
414
419
438
445
442
462
444


In [29]:
print(X_train.shape)
print(Y_train.shape)
print(attention_masks.shape)
print(rule_predictions.shape)


torch.Size([1780, 510])
torch.Size([1780])
torch.Size([1780, 510])
torch.Size([1780])


In [30]:
import os
import re
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, Trainer, TrainingArguments, AutoModel
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, balanced_accuracy_score
os.environ["WANDB_DISABLED"] = "true"


access_token = "TOKEN"

# Custom model class
class NeuroSymbolicClassifier(nn.Module):
    def __init__(self, model_name, num_labels, lambda_weight=0.1):
        super(NeuroSymbolicClassifier, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_name, token=access_token)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)
        self.lambda_weight = lambda_weight  # Weight for rule-based penalty

    def forward(self, input_ids, attention_mask=None, labels=None, rule_predictions=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)

        # Use the pooled output (e.g., CLS token) for classification
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        logits = self.classifier(pooled_output)

        # Calculate neural loss (cross-entropy loss)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            neural_loss = loss_fn(logits, labels)

            # Calculate rule-based penalty if rule_predictions are provided
            if rule_predictions is not None:
                rule_penalty = torch.mean((torch.argmax(logits, dim=1) != rule_predictions).float())
                total_loss = neural_loss + self.lambda_weight * rule_penalty
            else:
                total_loss = neural_loss

            loss = total_loss

        return {'loss': loss, 'logits': logits}


# Define a custom Dataset class
class DepressionDataset(Dataset):
    def __init__(self, input_ids, labels, attention_masks, rule_predictions):
        self.input_ids = input_ids
        self.labels = labels
        self.attention_masks = attention_masks
        self.rule_predictions = rule_predictions

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long).squeeze(),
            'rule_predictions': torch.tensor(self.rule_predictions[idx], dtype=torch.long)
        }

# Define the compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    bac = balanced_accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'balanced_accuracy': bac
    }

# Initialize KFold and metrics storage
kfold = KFold(n_splits=5, shuffle=True, random_state=17)
k_fold_iteration = 0
total_bac = []
total_precision = []
total_recall = []
total_f1 = []

trained_models_base_dir = '/content/drive/MyDrive/Fairness_NLP//Neurosymbolic_Classifier_5thDec'
model_name = "mental/mental-roberta-base"

# Perform k-fold cross-validation
for train_index, test_index in kfold.split(X_train):
    k_fold_iteration += 1

    # Split into training and validation sets for this fold
    train_inputs, val_inputs = X_train[train_index], X_train[test_index]
    train_labels, val_labels = Y_train[train_index], Y_train[test_index]
    attention_mask_train, attention_mask_val = attention_masks[train_index], attention_masks[test_index]
    rule_preds_train, rule_preds_val = rule_predictions[train_index], rule_predictions[test_index]

    # Initialize datasets for the current fold
    train_dataset = DepressionDataset(train_inputs, train_labels, attention_mask_train, rule_preds_train)
    val_dataset = DepressionDataset(val_inputs, val_labels, attention_mask_val, rule_preds_val)

    # Initialize the model
    model = NeuroSymbolicClassifier(model_name, num_labels=2, lambda_weight=0.1)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'{trained_models_base_dir}/trained_models/k_fold_{k_fold_iteration}/',
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        evaluation_strategy='epoch',
        learning_rate=2e-5,
        weight_decay=0.01,
        save_strategy='epoch',
        logging_dir=f'{trained_models_base_dir}/logs/k_fold_{k_fold_iteration}',
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        save_total_limit=1,
        logging_steps=1
    )

    # Create a Trainer for this fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the validation set and store metrics
    metrics = trainer.evaluate(val_dataset)
    total_bac.append(metrics['eval_balanced_accuracy'])
    total_precision.append(metrics['eval_precision'])
    total_recall.append(metrics['eval_recall'])
    total_f1.append(metrics['eval_f1'])

    print(f"Fold {k_fold_iteration} - Balanced Accuracy: {metrics['eval_balanced_accuracy']}, Precision: {metrics['eval_precision']}, Recall: {metrics['eval_recall']}, F1 Score: {metrics['eval_f1']}")

# Print overall results after cross-validation
print(f"Average Balanced Accuracy: {np.mean(total_bac)}, Average Precision: {np.mean(total_precision)}, Average Recall: {np.mean(total_recall)}, Average F1 Score: {np.mean(total_f1)}")

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Balanced Accuracy
1,0.633300,0.499553,0.792135,0.000000,0.000000,0.000000,0.500000
2,0.460700,0.502319,0.744382,0.473988,0.414141,0.554054,0.674190
3,0.078000,0.429956,0.817416,0.532374,0.569231,0.500000,0.700355
4,0.113400,0.727426,0.738764,0.513089,0.418803,0.662162,0.710514
5,1.058800,0.989501,0.783708,0.533333,0.483516,0.594595,0.713964
6,0.013800,1.332835,0.775281,0.545455,0.470588,0.648649,0.728580
7,0.025100,1.701722,0.764045,0.533333,0.452830,0.648649,0.721487
8,0.037600,1.499435,0.797753,0.544304,0.511905,0.581081,0.717846
9,0.012600,1.959134,0.747191,0.531250,0.432203,0.689189,0.725800
10,0.037600,1.759866,0.769663,0.544444,0.462264,0.662162,0.730017


Fold 1 - Balanced Accuracy: 0.700354609929078, Precision: 0.5692307692307692, Recall: 0.5, F1 Score: 0.5323741007194245


Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Balanced Accuracy
1,0.308700,0.498893,0.783708,0.172043,0.421053,0.108108,0.534551
2,0.575600,0.529741,0.733146,0.344828,0.352113,0.337838,0.587359
3,0.248600,0.443545,0.825843,0.446429,0.657895,0.337838,0.645869
4,0.435900,0.782137,0.766854,0.551351,0.459459,0.689189,0.738212
5,0.728900,0.846089,0.814607,0.592593,0.545455,0.648649,0.753402
6,0.025700,1.332050,0.750000,0.552764,0.440000,0.743243,0.747508
7,0.013100,1.449664,0.772472,0.562162,0.468468,0.702703,0.746741
8,0.012500,1.152080,0.839888,0.595745,0.626866,0.567568,0.739458
9,0.012600,1.360435,0.800562,0.569697,0.516484,0.635135,0.739553
10,0.025000,1.332742,0.806180,0.571429,0.528736,0.621622,0.738116


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Balanced Accuracy
1,0.371700,0.459780,0.794944,0.026667,1.000000,0.013514,0.506757
2,0.761300,0.482360,0.794944,0.562874,0.505376,0.635135,0.736007
3,0.252700,0.394479,0.831461,0.600000,0.592105,0.608108,0.749090
4,0.038500,0.760789,0.803371,0.602273,0.519608,0.716216,0.771229
5,0.662000,0.761381,0.837079,0.637500,0.593023,0.689189,0.782538
6,0.027600,1.100653,0.792135,0.606383,0.500000,0.770270,0.784071
7,0.012900,0.879763,0.853933,0.648649,0.648649,0.648649,0.778225
8,0.025000,0.942750,0.862360,0.652482,0.686567,0.621622,0.773577
9,0.037700,1.286137,0.811798,0.629834,0.532710,0.770270,0.796483
10,0.025100,1.153870,0.820225,0.627907,0.551020,0.729730,0.786851


Fold 4 - Balanced Accuracy: 0.7735767682576193, Precision: 0.6865671641791045, Recall: 0.6216216216216216, F1 Score: 0.6524822695035462


Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Balanced Accuracy
1,0.315800,0.513777,0.775281,0.000000,0.000000,0.000000,0.500000
2,0.174600,0.516218,0.792135,0.244898,0.666667,0.150000,0.564130
3,0.226300,0.553339,0.792135,0.421875,0.562500,0.337500,0.630707
4,0.470400,0.665488,0.792135,0.493151,0.545455,0.450000,0.670652
5,0.101000,0.834198,0.797753,0.506849,0.560606,0.462500,0.678714
6,0.038800,1.195717,0.783708,0.538922,0.517241,0.562500,0.705163
7,0.013700,1.358002,0.772472,0.490566,0.493671,0.487500,0.671286
8,0.609200,1.448234,0.780899,0.524390,0.511905,0.537500,0.694475
9,0.012800,1.597403,0.769663,0.528736,0.489362,0.575000,0.700543
10,0.025200,1.600947,0.780899,0.556818,0.510417,0.612500,0.721105


Fold 5 - Balanced Accuracy: 0.6787137681159421, Precision: 0.5606060606060606, Recall: 0.4625, F1 Score: 0.5068493150684932
Average Balanced Accuracy: 0.7080363990047085, Average Precision: 0.6373718818494938, Average Recall: 0.4970045045045045, Average F1 Score: 0.5495535652919976


In [ ]:
print(f"Average Balanced Accuracy: {np.mean(total_bac)}, Average Precision: {np.mean(total_precision)}, Average Recall: {np.mean(total_recall)}, Average F1 Score: {np.mean(total_f1)}")

NameError: name 'total_bac' is not defined